In [3]:
import pandas as pd
import deepchem as dc
from deepchem.models.torch_models.chemberta import Chemberta

c:\Users\BSH\miniconda3\envs\deepchem\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
train_df = pd.read_csv("./data/train.csv")
train_df.head()

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.259,117.37
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.172,73.47
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.585,62.45
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.475,92.60
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.337,42.43


In [9]:
loader = dc.data.CSVLoader(["HLM"], feature_field="SMILES", featurizer=dc.feat.DummyFeaturizer())
dataset = loader.create_dataset("data/train.csv")

In [17]:
pretrain_model_dir = "model/"
tokenizer_path = "seyonec/PubChem10M_SMILES_BPE_60k"
pretrain_model = Chemberta(task="regression", model_dir=pretrain_model_dir, tokenizer_path=tokenizer_path)
pretraining_loss = pretrain_model.fit(dataset, nb_epoch=1)

c:\Users\BSH\miniconda3\envs\deepchem\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\BSH\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


OutOfMemoryError: CUDA out of memory. Tried to allocate 148.00 MiB (GPU 0; 12.00 GiB total capacity; 9.90 GiB already allocated; 0 bytes free; 11.23 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# finetuning in regression mode
finetune_model_dir = "model"
finetune_model = Chemberta(task='regression', model_dir=finetune_model_dir, tokenizer_path=tokenizer_path)
finetune_model.load_from_pretrained(pretrain_model_dir)
finetuning_loss = finetune_model.fit(dataset, nb_epoch=1)